In [1]:
from google.cloud import storage
import pandas as pd
import io
import yfinance as yf
from datetime import datetime
import os
import time
import numpy as np

In [2]:
from google.cloud import storage
from google.oauth2 import service_account

In [3]:
import os

# Set the environment variable for the Google Cloud credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/workspaces/financial_data_scraping/jaber-financial-b20f23e23588.json'

# Verify that it has been set correctly
print("GOOGLE_APPLICATION_CREDENTIALS:", os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))


GOOGLE_APPLICATION_CREDENTIALS: /workspaces/financial_data_scraping/jaber-financial-b20f23e23588.json


In [4]:
from google.cloud import storage

def test_gcs_access(bucket_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    print(f"Bucket '{bucket_name}' exists: {bucket.exists()}")

# Replace 'your-bucket-name' with your actual bucket name
test_gcs_access('companies_details')


Bucket 'companies_details' exists: True


In [5]:
# Define Google Cloud Storage bucket name and file path
bucket_name = 'companies_details'
source_blob_name = 'data/companies_details.parquet'

def download_from_gcs(bucket_name, source_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(source_blob_name)
    file_content = blob.download_as_bytes()
    return file_content

# Download the file content from GCS
file_content = download_from_gcs(bucket_name, source_blob_name)

# Load the content into a pandas DataFrame
all_companies = pd.read_parquet(io.BytesIO(file_content))



In [6]:
# Function to fetch historical data
def fetch_historical_data(ticker, exchange_suffix):
    try:
        ticker_with_suffix = ticker + exchange_suffix  # Add dynamic suffix to the ticker
        data = yf.download(ticker_with_suffix, period="max")
        data = data.reset_index()  # Reset index to move date from index to column
        return data
    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}")
        return pd.DataFrame()

# Function to upload DataFrame to GCS
def upload_to_gcs(bucket_name, df, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    # Convert DataFrame to a BytesIO buffer
    buffer = io.BytesIO()
    df.to_parquet(buffer, index=False)
    buffer.seek(0)

    # Upload the buffer to GCS
    blob.upload_from_file(buffer, content_type='application/octet-stream')
    print(f"Data uploaded to {destination_blob_name} in bucket {bucket_name}.")

# Function to save DataFrame locally
def save_locally(df, local_path):
    df.to_parquet(local_path, index=False)
    print(f"Data saved locally to {local_path}.")

# Define bucket name and URL pattern
bucket_name = 'historical_data_details'
scraping_url = "https://finance.yahoo.com/quote/{ticker}/history/"  # Fixed URL pattern
scraping_timestamp = datetime.now().isoformat()  # Current timestamp
local_data_path = '/workspaces/financial_data_scraping/data/history/'

# Ensure the local directory exists
if not os.path.exists(local_data_path):
    os.makedirs(local_data_path)

# Loop through the 'Ticker' column in all_companies DataFrame
for idx, row in all_companies.iterrows():
    ticker = row['Ticker']
    isin = row['ISIN']
    exchange_suffix = row.get('Exchange_Suffix', '.MC')  # Get exchange suffix from the DataFrame or use default

    # Fetch historical data for the current ticker
    df = fetch_historical_data(ticker, exchange_suffix)

    # If data is retrieved, add additional columns
    if not df.empty:
        # Add ISIN, exchange, and ticker columns
        df['ISIN'] = isin
        df['Exchange'] = exchange_suffix.strip('.')  # Remove leading dot if present
        df['Ticker'] = ticker  # Save the ticker without the suffix

        # Add URL and timestamp columns
        df['scraping_url'] = scraping_url.format(ticker=ticker)
        df['scraping_timestamp'] = scraping_timestamp

        # Reorder columns to place 'Exchange', 'Ticker', 'ISIN' at the beginning
        columns_order = ['Ticker', 'Exchange', 'ISIN'] + [col for col in df.columns if col not in ['Ticker', 'Exchange', 'ISIN', 'scraping_url', 'scraping_timestamp']] + ['scraping_url', 'scraping_timestamp']
        df = df[columns_order]

        # Define local and GCS destination paths
        local_file_path = f"{local_data_path}{ticker}.parquet"
        gcs_blob_name = f"data/history/{ticker}.parquet"

        # Save DataFrame locally
        save_locally(df, local_file_path)

        # Upload DataFrame to GCS
        upload_to_gcs(bucket_name, df, gcs_blob_name)

        print(f"Processed historical data for {ticker}")

    # Introduce random delay between API calls (1 to 3 seconds)
    random_int = np.random.choice([1, 2, 3])
    time.sleep(random_int)

[*********************100%***********************]  1 of 1 completed


Data saved locally to /workspaces/financial_data_scraping/data/history/ANA.parquet.
Data uploaded to data/history/ANA.parquet in bucket historical_data_details.
Processed historical data for ANA


[*********************100%***********************]  1 of 1 completed


Data saved locally to /workspaces/financial_data_scraping/data/history/ACX.parquet.
Data uploaded to data/history/ACX.parquet in bucket historical_data_details.
Processed historical data for ACX


[*********************100%***********************]  1 of 1 completed


...

Data saved locally to /workspaces/financial_data_scraping/data/history/VID.parquet.
Data uploaded to data/history/VID.parquet in bucket historical_data_details.
Processed historical data for VID


[*********************100%***********************]  1 of 1 completed


Data saved locally to /workspaces/financial_data_scraping/data/history/VIS.parquet.
Data uploaded to data/history/VIS.parquet in bucket historical_data_details.
Processed historical data for VIS


[*********************100%***********************]  1 of 1 completed


Data saved locally to /workspaces/financial_data_scraping/data/history/VOC.parquet.
Data uploaded to data/history/VOC.parquet in bucket historical_data_details.
Processed historical data for VOC
